**IOC 17 – IP Spoofing**

OSI Layer 3 – Network Layer

**Cast of Characters (Starring in this Attack)**
Character	Role in the Attack
The Masquerader	The attacker — disguises their IP to appear trusted, bypassing controls or misleading logs
The Doorman	A firewall or network ACL — fooled into letting someone in based on IP identity
The Log Keeper	A SIEM or router log — records a fake IP, muddying attribution
The Target Host	Victim server or device — receives traffic from what looks like a trusted peer

**1. Attacker Analogy**
Imagine a con artist walking into a high-security building using a perfectly forged badge with someone else’s name on it — someone trusted by security. The guards (firewalls and systems) wave them in without question. The con artist may drop off a malicious package, take notes, or simply disappear — and the security logs show that the trusted person came and went.

That’s IP spoofing. The attacker forges the source IP address in network packets to impersonate another device. They may use this:

To bypass IP-based firewall rules

To evade detection or attribution

To trigger trust-based behavior on the receiving system (e.g., sending a reset to end a TCP connection). In certain advanced scenarios, an attacker may spoof the source IP address of a trusted machine — not to open a connection, but to interrupt one. Imagine two systems already communicating over a TCP session (say, a remote admin tool and a server). If the attacker spoofs the IP of one side of that connection and sends a specially crafted TCP RST (reset) packet, the recipient may accept it as legitimate — because it appears to be from the trusted peer. This instantly terminates the connection, and neither side knows it was an outsider who injected the command. This technique is used in TCP session hijacking, disruption attacks, or to silently sabotage communication without ever being part of the actual session. The trust is abused not by gaining access, but by spoofing identity and mimicking expected behavior — in this case, the act of gracefully closing a connection.

IP spoofing can stand alone or be part of a broader attack, such as DoS, man-in-the-middle, or reflection/amplification attacks.IP spoofing isn’t always the star of the show — sometimes it’s a supporting actor in larger attack sequences. In a Denial-of-Service (DoS) attack, spoofed IPs are used to disguise the attacker’s identity or to overload the target with reply traffic directed at innocent third parties. In man-in-the-middle (MITM) attacks, IP spoofing may be combined with ARP poisoning or routing deception to impersonate a trusted system, allowing the attacker to insert themselves between two devices. In reflection or amplification attacks (especially DDoS), IP spoofing is critical: the attacker sends small requests to misconfigured servers (like DNS or NTP) with the victim’s IP as the spoofed source, causing massive response data to flood the victim. In each case, IP spoofing is not the payload — it’s the deceptive envelope that makes the delivery possible.

**2. IOC (Observable Artifact), Telemetry Source, and Simulated Data**
IOC (Observable)	A TCP SYN packet claiming to originate from a trusted internal IP (e.g., 10.0.0.12), but with no return traffic and a TTL value inconsistent with internal hosts (e.g., TTL = 115), suggesting IP spoofing.
Primary Telemetry Source	Packet Capture (PCAP) — required to view IP header fields, including TTL and TCP flags
Pivot Telemetry Sources	NetFlow logs or Firewall logs — may reveal one-way traffic patterns or excessive SYNs that trigger deeper packet inspection
Key Technical Clue	TTL value does not match expected OS default (e.g., internal Linux hosts typically TTL = 64); session is never completed (SYN with no ACK)


PCAP Metadata: 

Frame 1423: 60 bytes on wire (480 bits), 60 bytes captured (480 bits)
Ethernet II, Src: 00:0c:29:3e:5c:1f, Dst: 00:50:56:c0:00:08
Internet Protocol Version 4, Src: 10.0.0.12, Dst: 192.0.2.15
    Version: 4
    Header Length: 20 bytes
    Time to live: 115
    Protocol: TCP (6)
    Header checksum: 0xa2b3 [correct]
    Source: 10.0.0.12
    Destination: 192.0.2.15
Transmission Control Protocol, Src Port: 34876, Dst Port: 80, Seq: 0, Len: 0
    Source Port: 34876
    Destination Port: 80
    Sequence Number: 0
    Header Length: 32 bytes
    Flags: 0x002 (SYN)
        000. .... = Reserved: Not set
        ...0 .... = Nonce: Not set
        .... 0... = CWR: Not set
        .... .0.. = ECN-Echo: Not set
        .... ..0. = Urgent: Not set
        .... ...0 = Acknowledgment: Not set
        .... ...1 = Synchronize: Set
        .... .... = FIN: Not set
    Window size value: 8192
    [No response seen to this packet]

    This packet is a legitimate TCP SYN with a spoofed source IP (10.0.0.12). The destination system (192.0.2.15) will attempt to reply with a SYN-ACK to that forged IP. However, in this packet capture, no response is visible. This may be due to factors such as asymmetric routing, limited capture scope, or sensor placement — not because the target failed to respond. The key indicators of spoofing in this packet are the forged internal source IP and the anomalous TTL value (115), which is inconsistent with local host defaults (Linux = 64, Windows = 128). Together, these suggest the SYN packet was generated by an external attacker using IP spoofing to conceal their true origin or manipulate trust-based firewall rules.





    Interpretation
This packet appears to originate from a trusted internal IP address (10.0.0.12), but several details strongly suggest IP spoofing:

The Time to Live (TTL) value is 115.

This is inconsistent with typical internal host behavior:

Linux systems usually default to TTL = 64

Windows systems usually default to TTL = 128

TTL = 115 implies the packet originated several hops away, which contradicts the idea that it came from a local machine using a private address.

The packet carries only a TCP SYN flag, initiating a connection to destination port 80 (HTTP), but:

There is no corresponding ACK or response, and

The capture tool explicitly notes: "[No response seen to this packet]"

Because the packet claims to be from a trusted internal address but behaves like external probe traffic, it likely represents a spoofed SYN packet used to bypass IP-based trust or participate in a scan, denial-of-service, or misdirection attempt.

Together, these traits indicate the packet is not genuine traffic from the IP it claims to be — but instead an artifact of IP spoofing, most likely generated with a custom packet crafting tool or an attacker-hosted script operating from outside the local network.


**3. Triage Framework Declaration**
Triage Type: Network Activity Indicator
Tools Required:

Nmap Scan – for identifying exposed systems and forged source behavior

Firewall Logs – to check rule bypass and spoofed packet entries

NetFlow Analysis – to examine packet flow and direction mismatches

Soft Interpretation (Network Triage)
In human terms, this is like an unknown visitor ringing your doorbell using your best friend’s voice through a speaker — they might get let in before anyone realizes it’s a trick. By using the identity of a trusted source, the attacker tricks systems into dropping their guard. Investigators rely on packet-level signs of deception, like missing reply paths or mismatched routing.

Summary Table – Expanded Triage Tools
Tool	Expanded Role in IP Spoofing Triage
Nmap	Simulates spoofed IP behavior, identifies exposed services, tests firewall trust assumptions
Firewall Logs	Detects rule bypass via spoofed internal IPs; shows SYNs without ACKs; logs TTL or geo-IP inconsistencies
NetFlow	Identifies one-way traffic, abnormal SYN-to-ACK ratios, repeated failures to establish sessions, TTL anomalies (if enriched)

Triage Source	Role	Priority
NetFlow	Statistical detection + trigger	Primary
Firewall Logs	Rule/path confirmation	Secondary
Nmap	Manual testing of exposure/trust assumptions	Tertiary

Simulated NetFlow Output – IP Spoofing

| Src IP     | Dst IP      | Protocol | Src Port | Dst Port | Packets | Bytes | Flags | Flow Direction | TTL  |
|------------|-------------|----------|----------|----------|---------|-------|--------|----------------|------|
| 10.0.0.12  | 192.0.2.15  | TCP      | 34876    | 80       | 1       | 60    | SYN    | Ingress        | 115  |
| 10.0.0.13  | 192.0.2.15  | TCP      | 34901    | 80       | 1       | 60    | SYN    | Ingress        | 112  |
| 10.0.0.14  | 192.0.2.15  | TCP      | 35000    | 80       | 1       | 60    | SYN    | Ingress        | 118  |
| 10.0.0.12  | 192.0.2.15  | TCP      | 34876    | 80       | 1       | 60    | SYN    | Ingress        | 115  |

NetFlow Interpretation
This NetFlow data shows multiple TCP SYN packets directed toward 192.0.2.15 (a likely public-facing web server) with no return traffic, indicating one-way flows. The source IPs (10.0.0.12, 10.0.0.13, 10.0.0.14) all fall within internal address ranges, yet the observed Time To Live (TTL) values (e.g., 112–118) are inconsistent with internal host defaults — suggesting these packets may have originated from outside the network and are using spoofed internal IPs to bypass firewall rules or obscure attribution. The repetition of the same spoofed IP (10.0.0.12) across multiple entries further supports the presence of automated spoofing behavior, likely part of a scan or prelude to a denial-of-service attack. The absence of completed sessions (no SYN-ACKs or ACKs) and the alignment of packet structure across multiple flows make this pattern highly indicative of IP spoofing activity.




**4. Host OS Layer Mapping**
Not applicable — IP spoofing occurs purely at the network layer, with no host-based compromise unless the attack succeeds and leads to a payload delivery or secondary action.

**5. Cross-Layer Interaction Pivots**
If successful, may pivot from OSI Layer 3 (Network) to:

Layer 4 (Transport): Initiating or interrupting sessions. At Layer 4, if many spoofed SYN packets are sent, they can initiate large numbers of half-open TCP sessions, leading to exhaustion of connection queues and interrupting legitimate transport-layer services. 


Host-based triage is only needed if additional activity beyond the spoofed SYN is observed — such as full session establishment or suspicious inbound payloads.
OSI Layer	Role

Layer 3 – Network	Core of the attack — forged source IP in IPv4 header
Layer 4 – Transport	Secondary effect — used to manipulate sessions or evade logging

**7. Attacker Behavior Interpretation**
In this scenario, the attacker is performing deception at the identity level by forging the source IP address in the IP header of a TCP SYN packet. Their objective is not to establish a session, but to disguise their origin and impersonate a trusted internal host. By spoofing the source IP, the attacker causes the target system to respond — but the response is sent to a forged address, not to the attacker, leaving the attacker hidden and untraceable via normal means.

This behavior is typically used for stealth reconnaissance, firewall rule testing, or attribution obfuscation. Rather than observing replies, the attacker infers system behavior based on whether their spoofed packets are allowed through security controls or recorded in logs. It is not intended to generate a dialogue with the target but to trigger system-level behavior without revealing the attacker’s true IP.

Because the TCP handshake is never completed, the attack often leaves behind network-layer artifacts such as:

Half-open TCP sessions (if the spoofed IP exists and does not respond)

One-way NetFlow records

Abnormal TTL values

Internal IP addresses appearing on external ingress points

These patterns make the activity detectable to a skilled analyst, even if the attacker remains operationally hidden.


**8. Defender Action Summary**

The primary defender response begins with NetFlow analysis, which offers the most scalable and accessible insight into spoofed IP activity. Analysts should look for:

One-way TCP flows with only SYN flags and no response packets

Repeated connection attempts with no session establishment

Source IPs that belong to internal ranges (e.g., 10.x.x.x) showing up at external ingress points

Abnormal TTL values (e.g., 115+) if the NetFlow system includes enriched IP header data (IPFIX)

Once NetFlow reveals a suspicious pattern, the investigation can pivot to firewall logs to confirm whether these spoofed packets bypassed IP-based controls. Analysts should check:

Whether spoofed internal IPs were allowed through based on source trust

If the firewall silently dropped the connections or passed them to internal systems

Whether routing paths or interface entry points are consistent with policy expectations

If confirmation is needed or if firewall policy evasion is suspected, defenders may use Nmap in a controlled test environment to simulate traffic using forged source IPs and observe how the system responds. This helps test whether ACLs or firewall rules are over-trusting of IP-based identity.

Finally, if the spoofed IP traffic is occurring at volume, defenders should also consider whether the activity is part of a SYN flood (transport-layer DoS) or a reflection/amplification attack. In those cases:

Use packet capture to examine packet size, response volume, and session queue behavior

Look for signs that third-party servers (e.g., DNS or NTP) are being triggered to send replies to a spoofed victim IP

Identify amplification sources by analyzing the destination services and packet response ratios


**9. Attacker Strategy Notes**
The attacker’s strategy in this scenario is based on identity deception without interaction. By forging TCP SYN packets with spoofed internal source IPs, the attacker avoids receiving replies entirely — but still triggers behavioral responses from the network. The attacker isn’t trying to establish a session or deliver a payload at this stage. Their goal is to bypass basic source-IP-based controls, confuse defenders, or map firewall rules by appearing as a trusted internal host.

This tactic is stateless and unidirectional, allowing the attacker to operate without exposure — especially if they rotate spoofed IPs or blend their probes with legitimate traffic patterns. In a more advanced variant, the attacker might scale this approach into a SYN flood to exhaust session tables or use spoofed requests to third-party services to create amplified denial-of-service via reflection — though neither of those escalations are evidenced in this IOC.

The strength of this approach lies in its deniability and evasion: no connection is completed, no content is delivered, and no reverse path exists for retaliation. But its weakness is detectability through pattern analysis: NetFlow records, TTL inconsistencies, and incomplete handshakes leave behind a trail of low-level anomalies that a well-instrumented network can detect.

Ultimately, the attacker is not trying to win with sophistication — they’re testing whether defenders are paying attention to what shouldn’t be happening, even when nothing seems to happen at all.

Recon Value Is Low

The attacker may not even know if the spoofed IP exists or is routable.
The SYN isn’t acknowledged, so no learning happens.
Correct. This technique doesn’t yield actionable intel unless the attacker has additional capabilities (e.g., MITM, response sniffing, or internal foothold).

Spoofed Traffic Did Enter the Network

The only “compromise” is that a forged packet got through — an ingress failure.
Exactly. This is not a host compromise — it’s a trust boundary violation at the network perimeter.

Standalone Value Is Limited

Unless paired with a SYN flood, session hijack, or internal access, this attack has no meaningful impact on its own.
Yes. This is a low-signal, low-yield probe unless it’s part of a larger campaign.

What This IOC Is:
A low-noise, low-effort deception meant to:

Test trust boundaries

Confuse attribution

Serve as a precursor to more meaningful action

What This IOC Is Not:
It’s not evidence of intrusion

It’s not data exfiltration

It’s not host compromise

It’s not even successful recon without context


**10. Who’s Who – Object Role Clarification**
Object	Role in Attack
IP Header (Source IP)	The forged field used to misrepresent the sender
Firewall / ACL	Bypassed due to trusting the forged source address
Packet Capture (PCAP)	Used to detect anomalies: TTLs, half-open handshakes, unreachable return paths
NetFlow / Flow Logs	Used to detect flow direction inconsistencies
SIEM Alert	May trigger based on policy violations or spoof detection heuristics


**Conclusion / Analyst README**

This case study examined a forged TCP SYN packet using a spoofed internal IP address as its source. The attacker did not attempt to complete a session or deliver a payload — instead, they employed identity deception at the IP layer, crafting traffic designed to resemble trusted internal communication.

The activity did not escalate to a transport or application-layer interaction. No system compromise occurred, and the spoofed packet was not acknowledged from the attacker’s side. This IOC represents a network-layer ingress failure, where trust was implied by IP alone, not validated by handshake or authentication.

From a defensive perspective, this scenario reinforces the importance of:

Flow-level visibility using tools like NetFlow or IPFIX

Awareness of one-way TCP activity

Detecting internal IPs arriving from external interfaces

Contextual analysis of TTL anomalies and incomplete sessions

While IP spoofing is not inherently damaging on its own, it can support broader campaigns — including evasion of IP-based ACLs, trust probing, or transport-layer denial-of-service attacks. Recognizing and interpreting this type of low-yield, low-noise activity is part of a broader analytical skillset that underpins real-world intrusion detection and exam-aligned incident response logic.


